In [1]:
####### IMPORTS
import pandas as pd
import geopandas
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import shapely.geometry
from IPython.core.display import display, HTML
from shapely.geometry import Polygon
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\rmpjazz\AppData\Local\Temp\ipykernel_10984\69770969.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
####### IMPORT DATA
cell_geo = pd.read_pickle("./dataframes/cell_geometry")
cell = geopandas.GeoDataFrame(cell_geo, columns=['geometry'], 
                                 crs='epsg:4326')

jsonfile = ('./mygeodata/fire_station_point.geojson')
fire_station_points = geopandas.read_file(jsonfile)

jsonfile = ('./ark28722-s73w23-geojson.json')
world = geopandas.read_file(jsonfile)

fires = pd.read_pickle("./dataframes/all_fires")
fires = fires[~fires["GIS_ACRES"].isnull()]
fires = fires[~fires.duplicated()]
fires = fires[~fires[["FIRE_NAME", "INC_NUM", "ALARM_DATE", "CONT_DATE", "CAUSE", "geometry"]].duplicated()]

fires = fires.loc[(fires["YEAR"] >= 2020) & (fires["YEAR"] <= 2023) & (fires["MONTH"] >= 6) & (fires["MONTH"] <= 10)]



In [3]:
####### PLOTTING
import matplotlib.colors as colors

def plotter(dataset):
    cell["interpolated_value"] = dataset
    ax = cell.plot(column='interpolated_value', figsize=(10, 10), cmap='plasma', legend=True)
    world.to_crs(cell.crs).plot(ax = ax, color = "none", edgecolor = 'black', figsize = (8,8))
    return ax
def log_plotter(dataset):
    cell["interpolated_value"] = dataset
    ax = cell.plot(column='interpolated_value', figsize=(10, 10), cmap='plasma', norm=colors.SymLogNorm(vmin=cell["interpolated_value"].min(), vmax=cell["interpolated_value"].max(), linthresh = 0.02), legend=True)
    world.to_crs(cell.crs).plot(ax = ax, color = "none", edgecolor = 'black', figsize = (8,8))


In [4]:
###### FIRESTATION POINT TO CELL
n_cells=828

cell_bounds = {}
for j in cell.index:
    cell_bounds[j] = cell["geometry"][j].bounds

positions_cell = [(int((cell["geometry"][i].centroid.x+125)*n_cells/12), int((cell["geometry"][i].centroid.y-32)*n_cells/12)) for i in cell.index]
cell["position"] = positions_cell
cell = cell.sort_values(by=["position"])

cell_size = (12)/n_cells
positions_fs = [(int((fire_station_points["geometry"][i].x+125)*n_cells/12), fire_station_points["geometry"][i].y) for i in fire_station_points.index]
fire_station_points["position"] = positions_fs
fire_station_points = fire_station_points.sort_values(by=["position"])

ind = {}
indj = 0
j = cell.index[indj]
for i in tqdm(fire_station_points.index):
    xc = fire_station_points["geometry"][i].x
    yc = fire_station_points["geometry"][i].y
    
    start = indj
    broken = False
    while not(cell_bounds[j][0] <= xc and cell_bounds[j][1] <= yc and cell_bounds[j][2] >= xc and cell_bounds[j][3] >= yc):
        indj += 1
        j = cell.index[indj]
        
        if (cell["position"][j][0] > fire_station_points["position"][i][0]):
            indj = start
            j = cell.index[indj]
            broken = True
            break
    if (broken):
        ind[i] = np.nan
    else:
        ind[i] = j
# null_firestations = []
# for x in ind.keys():
#     if (ind[x] is np.nan):
#         null_firestations.append(x)
# for i in fire_station_points.index:
#     if (np.isnan(ind[i]))
# fire_station_points = fire_station_points[[(not (i in null_firestations)) for i in fire_station_points.index]]


  0%|          | 0/1143 [00:00<?, ?it/s]

In [5]:
def check1(pos, size_coverage):
    score = 0
    for row in range(size_coverage):
        for column in range(size_coverage):
            new_ind = pos+column+row*760
            if new_ind in cell.index:
                score += cell["interpolated_value"][new_ind]
    return (score, 1)
def check2(pos, size_coverage):
    score = 0
    for row in range(size_coverage):
        for column in range(size_coverage):
            new_ind = pos-column+row*760
            if new_ind in cell.index:
                score += cell["interpolated_value"][new_ind]
    return (score, 2)
def check3(pos, size_coverage):
    score = 0
    for row in range(size_coverage):
        for column in range(size_coverage):
            new_ind = pos-column-row*760
            if new_ind in cell.index:
                score += cell["interpolated_value"][new_ind]
    return (score, 3)
def check4(pos, size_coverage):
    score = 0
    for row in range(size_coverage):
        for column in range(size_coverage):
            new_ind = pos+column-row*760
            if new_ind in cell.index:
                score += cell["interpolated_value"][new_ind]
    return (score, 4)
def check5(pos, size_coverage):
    score = 0
    for row in range(-(size_coverage//2), size_coverage//2+1):
        for column in range(size_coverage):
            new_ind = pos+column+row*760
            if new_ind in cell.index:
                score += cell["interpolated_value"][new_ind]
    return (score, 5)
def check6(pos, size_coverage):
    score = 0
    for row in range(-(size_coverage//2), size_coverage//2+1):
        for column in range(size_coverage):
            new_ind = pos-column+row*760
            if new_ind in cell.index:
                score += cell["interpolated_value"][new_ind]
    return (score, 6)
def check7(pos, size_coverage):
    score = 0
    for row in range(size_coverage):
        for column in range(-(size_coverage//2), size_coverage//2+1):
            new_ind = pos+column-row*760
            if new_ind in cell.index:
                score += cell["interpolated_value"][new_ind]
    return (score, 7)
def check8(pos, size_coverage):
    score = 0
    for row in range(size_coverage):
        for column in range(-(size_coverage//2), size_coverage//2+1):
            new_ind = pos+column+row*760
            if new_ind in cell.index:
                score += cell["interpolated_value"][new_ind]
    return (score, 8)
def check9(pos, size_coverage):
    score = 0
    for row in range(-(size_coverage//2), size_coverage//2+1):
        for column in range(-(size_coverage//2), size_coverage//2+1):
            new_ind = pos+column-row*760
            if new_ind in cell.index:
                score += cell["interpolated_value"][new_ind]
    return (score, 9)      

def check(pos, di, size_coverage):
#     print(pos, di)
#     return (cell["interpolated_value"][pos], di, cell["geometry"][pos])
    if (di == 1):
        return check1(pos, size_coverage)
    if (di == 2):
        return check2(pos, size_coverage)
    if (di == 3):
        return check3(pos, size_coverage)
    if (di == 4):
        return check4(pos, size_coverage)
    if (di == 5):
        return check5(pos, size_coverage)
    if (di == 6):
        return check6(pos, size_coverage)
    if (di == 7):
        return check7(pos, size_coverage)
    if (di == 8):
        return check8(pos, size_coverage)
    if (di == 9):
        return check9(pos, size_coverage)
def null1(pos, size_coverage):
    score = 0
    for row in range(size_coverage):
        for column in range(size_coverage):
            new_ind = pos+column+row*760
            cell.loc[new_ind,"interpolated_value"] = 0
def null2(pos, size_coverage):
    score = 0
    for row in range(size_coverage):
        for column in range(size_coverage):
            new_ind = pos-column+row*760
            cell.loc[new_ind,"interpolated_value"] = 0
def null3(pos, size_coverage):
    score = 0
    for row in range(size_coverage):
        for column in range(size_coverage):
            new_ind = pos-column-row*760
            cell.loc[new_ind,"interpolated_value"] = 0
def null4(pos, size_coverage):
    score = 0
    for row in range(size_coverage):
        for column in range(size_coverage):
            new_ind = pos+column-row*760
            cell.loc[new_ind,"interpolated_value"] = 0
def null5(pos, size_coverage):
    score = 0
    for row in range(-(size_coverage//2), size_coverage//2+1):
        for column in range(size_coverage):
            new_ind = pos+column+row*760
            cell.loc[new_ind,"interpolated_value"] = 0
def null6(pos, size_coverage):
    score = 0
    for row in range(-(size_coverage//2), size_coverage//2+1):
        for column in range(size_coverage):
            new_ind = pos-column+row*760
            cell.loc[new_ind,"interpolated_value"] = 0
def null7(pos, size_coverage):
    score = 0
    for row in range(size_coverage):
        for column in range(-(size_coverage//2), size_coverage//2+1):
            new_ind = pos+column-row*760
            cell.loc[new_ind,"interpolated_value"] = 0
def null8(pos, size_coverage):
    score = 0
    for row in range(size_coverage):
        for column in range(-(size_coverage//2), size_coverage//2+1):
            new_ind = pos+column+row*760
            cell.loc[new_ind,"interpolated_value"] = 0
def null9(pos, size_coverage):
    score = 0
    for row in range(-(size_coverage//2), size_coverage//2+1):
        for column in range(-(size_coverage//2), size_coverage//2+1):
            new_ind = pos+column-row*760
            cell.loc[new_ind,"interpolated_value"] = 0

def null(pos, di, size_coverage):
    if (di == 1):
        null1(pos, size_coverage)
    if (di == 2):
        null2(pos, size_coverage)
    if (di == 3):
        null3(pos, size_coverage)
    if (di == 4):
        null4(pos, size_coverage)
    if (di == 5):
        null5(pos, size_coverage)
    if (di == 6):
        null6(pos, size_coverage)
    if (di == 7):
        null7(pos, size_coverage)
    if (di == 8):
        null8(pos, size_coverage)
    if (di == 9):
        null9(pos, size_coverage)

In [6]:
def go_enhanced(coverage_numbers, size_coverage):
    coverage_numbers.sort()
    cell_size = (12)/n_cells
    cell["interpolated_value"] = dataframe.copy()
    
    ##### THIS EXISTS BTW
    cell["interpolated_value"] = cell["interpolated_value"].replace(np.nan, 0)

    all_coverages = []

    for i in fire_station_points.index:

        pos = ind[i]
        
        if (np.isnan(pos)):
            continue
        
        for diit in range(1, 10):
            score, di = check(pos, diit, size_coverage)
            all_coverages.append((score, pos, diit, i))

    all_coverages.sort()

    ret = []
    selected_coverages = []
    summed_coverage = 0
    mex = max(coverage_numbers)
#     print(all_coverages)
    for i in tqdm(range(mex)):
        if (i in coverage_numbers):
            ret.append((summed_coverage, selected_coverages.copy()))
        summed_coverage += all_coverages[-1][0]
#         print(summed_coverage)
        selected_coverages.append(all_coverages[-1])
        null(all_coverages[-1][1], all_coverages[-1][2], size_coverage)
        for coverage_num in range(len(all_coverages)):
            score, di = check(all_coverages[coverage_num][1], all_coverages[coverage_num][2], size_coverage)
    #         print(all_coverages[coverage_num])
    #         print((score, all_coverages[coverage_num][1], di, all_coverages[coverage_num][3]))
    #         print()
            all_coverages[coverage_num] = (score, all_coverages[coverage_num][1], di, all_coverages[coverage_num][3])

        all_coverages.sort()
    ret.append((summed_coverage, selected_coverages.copy()))
    return ret

In [7]:
# def countcovered(coverages, size):
#     totalcovered = 0
#     ansar = np.array([False for i in range(len(fires))])
#     for x in coverages:
#         indx = x[1]
#         minx, miny, maxx, maxy = cell["geometry"][indx].bounds
#         if (x[2] == 1):
#             pos = (minx, miny)
#         if (x[2] == 2):
#             pos = (minx, maxy-size*12/n_cells)
#         if (x[2] == 3):
#             pos = (maxx-size*12/n_cells, maxy-size*12/n_cells)
#         if (x[2] == 4):
#             pos = (maxx-size*12/n_cells, miny)
#         if (x[2] == 5):
#             pos = (minx-(size//2)*12/n_cells, miny)
#         if (x[2] == 6):
#             pos = (minx-(size//2)*12/n_cells, maxy-size*12/n_cells)
#         if (x[2] == 7):
#             pos = (maxx-size*12/n_cells, miny-(size//2)*12/n_cells)
#         if (x[2] == 8):
#             pos = (minx, miny-(size//2)*12/n_cells)
#         if (x[2] == 9):
#             pos = (minx-(size//2)*12/n_cells, miny-(size//2)*12/n_cells)
#         polygon = Polygon([(pos[0], pos[1]), (pos[0] + size*12/n_cells, pos[1]), (pos[0] + size*12/n_cells, pos[1] + size*12/n_cells), (pos[0], pos[1] + size*12/n_cells)])
# #         coverage_bounds = (pos[0], pos[1], , pos[1] + 5*12/n_cells)
# #         print(polygon)
#         ansar = np.logical_or(ansar, fires.overlaps(polygon))
#         ansar = np.logical_or(ansar, fires.within(polygon))
#         ansar = np.logical_or(ansar, fires.contains(polygon))
#     return ansar.sum()

def countcovered_enhanced(coverages, size):
    totalcovered = 0
    ansar = np.array([False for i in range(len(fires))])
    res = [0]
    for x in coverages:
        indx = x[1]
        minx, miny, maxx, maxy = cell["geometry"][indx].bounds
        if (x[2] == 1):
            pos = (minx, miny)
        if (x[2] == 2):
            pos = (minx, maxy-size*12/n_cells)
        if (x[2] == 3):
            pos = (maxx-size*12/n_cells, maxy-size*12/n_cells)
        if (x[2] == 4):
            pos = (maxx-size*12/n_cells, miny)
        if (x[2] == 5):
            pos = (minx-(size//2)*12/n_cells, miny)
        if (x[2] == 6):
            pos = (minx-(size//2)*12/n_cells, maxy-size*12/n_cells)
        if (x[2] == 7):
            pos = (maxx-size*12/n_cells, miny-(size//2)*12/n_cells)
        if (x[2] == 8):
            pos = (minx, miny-(size//2)*12/n_cells)
        if (x[2] == 9):
            pos = (minx-(size//2)*12/n_cells, miny-(size//2)*12/n_cells)
        polygon = Polygon([(pos[0], pos[1]), (pos[0] + size*12/n_cells, pos[1]), (pos[0] + size*12/n_cells, pos[1] + size*12/n_cells), (pos[0], pos[1] + size*12/n_cells)])
#         coverage_bounds = (pos[0], pos[1], , pos[1] + 5*12/n_cells)
#         print(polygon)
        ansar = np.logical_or(ansar, fires.overlaps(polygon))
        ansar = np.logical_or(ansar, fires.within(polygon))
        ansar = np.logical_or(ansar, fires.contains(polygon))
        res.append(ansar.sum())
    return res

# def countcoveredacres(coverages, size):
#     totalcovered = 0
#     ansar = np.array([False for i in range(len(fires))])
#     for x in coverages:
#         indx = x[1]
#         minx, miny, maxx, maxy = cell["geometry"][indx].bounds
#         if (x[2] == 1):
#             pos = (minx, miny)
#         if (x[2] == 2):
#             pos = (minx, maxy-size*12/n_cells)
#         if (x[2] == 3):
#             pos = (maxx-size*12/n_cells, maxy-size*12/n_cells)
#         if (x[2] == 4):
#             pos = (maxx-size*12/n_cells, miny)
#         if (x[2] == 5):
#             pos = (minx-(size//2)*12/n_cells, miny)
#         if (x[2] == 6):
#             pos = (minx-(size//2)*12/n_cells, maxy-size*12/n_cells)
#         if (x[2] == 7):
#             pos = (maxx-size*12/n_cells, miny-(size//2)*12/n_cells)
#         if (x[2] == 8):
#             pos = (minx, miny-(size//2)*12/n_cells)
#         if (x[2] == 9):
#             pos = (minx-(size//2)*12/n_cells, miny-(size//2)*12/n_cells)
#         polygon = Polygon([(pos[0], pos[1]), (pos[0] + size*12/n_cells, pos[1]), (pos[0] + size*12/n_cells, pos[1] + size*12/n_cells), (pos[0], pos[1] + size*12/n_cells)])
# #         coverage_bounds = (pos[0], pos[1], , pos[1] + 5*12/n_cells)
# #         print(polygon)
#         ansar = np.logical_or(ansar, fires.overlaps(polygon))
#         ansar = np.logical_or(ansar, fires.within(polygon))
#         ansar = np.logical_or(ansar, fires.contains(polygon))
#     return np.nansum(fires["GIS_ACRES"][ansar])

def countcoveredacres_enhanced(coverages, size):
    totalcovered = 0
    ansar = np.array([False for i in range(len(fires))])
    res = [0]
    for x in coverages:
        indx = x[1]
        minx, miny, maxx, maxy = cell["geometry"][indx].bounds
        if (x[2] == 1):
            pos = (minx, miny)
        if (x[2] == 2):
            pos = (minx, maxy-size*12/n_cells)
        if (x[2] == 3):
            pos = (maxx-size*12/n_cells, maxy-size*12/n_cells)
        if (x[2] == 4):
            pos = (maxx-size*12/n_cells, miny)
        if (x[2] == 5):
            pos = (minx-(size//2)*12/n_cells, miny)
        if (x[2] == 6):
            pos = (minx-(size//2)*12/n_cells, maxy-size*12/n_cells)
        if (x[2] == 7):
            pos = (maxx-size*12/n_cells, miny-(size//2)*12/n_cells)
        if (x[2] == 8):
            pos = (minx, miny-(size//2)*12/n_cells)
        if (x[2] == 9):
            pos = (minx-(size//2)*12/n_cells, miny-(size//2)*12/n_cells)
        polygon = Polygon([(pos[0], pos[1]), (pos[0] + size*12/n_cells, pos[1]), (pos[0] + size*12/n_cells, pos[1] + size*12/n_cells), (pos[0], pos[1] + size*12/n_cells)])
#         coverage_bounds = (pos[0], pos[1], , pos[1] + 5*12/n_cells)
#         print(polygon)
        ansar = np.logical_or(ansar, fires.overlaps(polygon))
        ansar = np.logical_or(ansar, fires.within(polygon))
        ansar = np.logical_or(ansar, fires.contains(polygon))
        res.append(np.nansum(fires["GIS_ACRES"][ansar]))
    return res


In [8]:
dataframe_name = "./dataframes/2000-2019/" + str("linear_interpolation_log")
dataframe = pd.read_pickle(dataframe_name)
cell["interpolated_value"] = dataframe.copy()

In [9]:
np.max(dataframe)

11.802599479731168

In [ ]:
import os
import pickle
interpolations = ["linear_interpolation_log", "linear_interpolation_log_shifted10000", "linear_interpolation_normal", #SEE HOW MUCH SHIFTED IS BEST
                  "nearest_interpolation_log", "nearest_interpolation_log_shifted10000", "nearest_interpolation_normal",
                  "inverse_distance_weighted_log", "inverse_distance_weighted_log_shifted10000", "inverse_distance_weighted_normal", "inverse_distance_weighted_power_0.2_normal",
                  "cubic_interpolation_log", "cubic_interpolation_normal",
                  "gaussian_kde", 
                  "universal_kriging_gaussian_regional_linear_drift_values_log", "universal_kriging_linear_regional_linear_drift_values_log",
                  "ordinary_kriging_gaussian_geographic_nlags_10_values_log", "ordinary_kriging_linear_geographic_nlags_10_values_log",
                  "universal_kriging_gaussian_regional_linear_drift_values_log_shifted10000", # don't have time to generate this one mb "universal_kriging_linear_regional_linear_drift_values_log_shifted10000"
                  "ordinary_kriging_gaussian_geographic_nlags_10_values_log_shifted10000", "ordinary_kriging_linear_geographic_nlags_10_values_log_shifted10000",
                 ]

max_drones = 1000
step_size = 1
for size in [1,3,5]:
    for interpchoose in interpolations:
        savedir = os.path.join(os.getcwd(), 'tradeoff_results', str(size)+'x' + str(size))
        if (not os.path.exists(savedir)):
            os.mkdir(savedir)
        if (os.path.exists(os.path.join(savedir,interpchoose))):
            continue
        dataframe_name = "./dataframes/2000-2019/" + str(interpchoose)
        dataframe = pd.read_pickle(dataframe_name)
        cell["interpolated_value"] = dataframe.copy()
        
        drone_nums = [i for i in range(0, max_drones+1, step_size)]
        go_enh_results = go_enhanced(drone_nums, size)
        x = drone_nums
        y = []
        y2 = countcovered_enhanced(go_enh_results[-1][1], size)
        y3 = countcoveredacres_enhanced(go_enh_results[-1][1], size)
        for j in go_enh_results:
            y.append(j[0])
        
        with open(os.path.join(savedir,interpchoose), 'wb') as fp:
            pickle.dump((drone_nums, y, y2, y3, go_enh_results), fp)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
import random
def go_rand_enhanced(coverage_numbers, size_coverage):
#     coverage_numbers.sort()
#     cell_size = (12)/n_cells
#     cell["interpolated_value"] = dataframe.copy()
    
#     ##### THIS EXISTS BTW
#     cell["interpolated_value"] = cell["interpolated_value"].replace(np.nan, 0)

#     all_coverages = []

#     for i in fire_station_points.index:

#         pos = ind[i]
        
#         if (np.isnan(pos)):
#             continue
        
#         for diit in range(1, 10):
#             score, di = check(pos, diit, size_coverage)
#             all_coverages.append((score, pos, diit, i))

#     all_coverages.sort()

#     ret = []
#     selected_coverages = []
#     summed_coverage = 0
#     mex = max(coverage_numbers)
# #     print(all_coverages)
#     for i in tqdm(range(mex)):
#         if (i in coverage_numbers):
#             ret.append((summed_coverage, selected_coverages.copy()))
        
#         summed_coverage += all_coverages[random_ind][0]
# #         print(summed_coverage)
#         selected_coverages.append(all_coverages[-1])
#         null(all_coverages[-1][1], all_coverages[-1][2], size_coverage)
#         all_coverages.pop(0)
#         for coverage_num in range(max(-len(all_coverages),-2*mex), 0):
#             score, di = check(all_coverages[coverage_num][1], all_coverages[coverage_num][2], size_coverage)
#     #         print(all_coverages[coverage_num])
#     #         print((score, all_coverages[coverage_num][1], di, all_coverages[coverage_num][3]))
#     #         print()
#             all_coverages[coverage_num] = (score, all_coverages[coverage_num][1], di, all_coverages[coverage_num][3])

#         all_coverages.sort()
#     ret.append((summed_coverage, selected_coverages.copy()))
#     return ret
    coverage_numbers.sort()
    cell_size = (12)/n_cells
    cell["interpolated_value"] = dataframe.copy()
    
    ##### THIS EXISTS BTW
    cell["interpolated_value"] = cell["interpolated_value"].replace(np.nan, 0)

    all_coverages = []

    for i in fire_station_points.index:

        pos = ind[i]
        
        if (np.isnan(pos)):
            continue
        
        for diit in range(1, 10):
            score, di = check(pos, diit, size_coverage)
            all_coverages.append((score, pos, diit, i))

    ret = []
    selected_coverages = []
    summed_coverage = 0
    mex = max(coverage_numbers)
#     print(all_coverages)
    for i in tqdm(range(mex)):
        if (i in coverage_numbers):
            ret.append((summed_coverage, selected_coverages.copy()))
        random_ind = random.randint(0, len(all_coverages)-1)
#         print(summed_coverage)
        selected_coverages.append(all_coverages[random_ind])
    ret.append((summed_coverage, selected_coverages.copy()))
    return ret

In [ ]:
import os
import pickle
max_drones = 1000
step_size = 1
for size in [1,3,5]:
    for randrun in range(10):
        savedir = os.path.join(os.getcwd(), 'tradeoff_results', str(size)+'x' + str(size))
        if (not os.path.exists(savedir)):
            os.mkdir(savedir)
        
        drone_nums = [i for i in range(0, max_drones+1, step_size)]
        go_enh_results = go_rand_enhanced(drone_nums, size)
        x = drone_nums
        y = []
        y2 = countcovered_enhanced(go_enh_results[-1][1], size)
        y3 = countcoveredacres_enhanced(go_enh_results[-1][1], size)
        for j in tqdm(go_enh_results):
            y.append(j[0])
        
        with open(os.path.join(savedir,"rand"+randrun), 'wb') as fp:
            pickle.dump((drone_nums, y, y2, y3, go_enh_results), fp)